In [ ]:
import numpy as np
import pandas as pd
from data.classes import *
import re
import os
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from Labeller import Labeller
import os
from bs4 import BeautifulSoup
import urllib.request as img_request

In [ ]:
cwd = os.getcwd()
data_dir = "data/"
img_dir = "E:/data/images/"

In [ ]:
url_name_image_tag = re.compile(r'data-link=\"(https://www\.boat24\.com/en/sailingboats/.{1,100}?\d\/)\"[^¤]*?title=\"(.{1,40})\"><div class=\"blurb__image-area\".+?srcset=\"(https://static\.b24\.co/fotos.+?\.jpg)')
image_tag = re.compile(r'src=\"(https://static\.b24\.co/fotos.+?\.jpg)')

In [ ]:
boat24_data_raw = pd.read_csv(os.path.join(data_dir, "boat24_data_raw.csv"))
sailboatdata_data = pd.read_csv(os.path.join(data_dir, "boat_data_clean.csv"))


for i in tqdm(range(0,boat24_data_raw.__len__())):
    found , name , distance = labeller.add_label(boat24_data_raw['name'][i])
    if found and not (boat24_data_raw['url'][i] == already_scraped['url']).any():
        images = re.findall(image_tag , boat24_data_raw['html'][i])
        print(Name_Classes[name])
        for image in images:
            file_name = "boat24_"+str(counter).zfill(5)+".jpg"
            img_path = os.path.join(img_dir,"boat24" , file_name)
            img_request.urlretrieve(image,img_path)
            row = sailboatdata_data[sailboatdata_data['name'] == name].iloc[0].to_frame().T
            row['img_path'] = file_name
            row['img_url'] = image
            boat24_data_clean = pd.concat([boat24_data_clean , row])
            counter += 1
boat24_data_clean.to_csv(os.path.join(data_dir,"boat24_data_clean.csv") , index=False)
boat24_data_raw['url'].to_csv(os.path.join(data_dir,"already_scraped.csv") , index=False)


In [ ]:
data_dir = os.path.join(os.getcwd(), "data")

# Sailboatdata

In [ ]:
pd.set_option('display.max_columns', None)
data_raw = pd.read_csv("boat_data.csv")
data_raw

In [ ]:
units_cleaner_tag = re.compile(r'([\d.]+)(\s*m|\s*kg)(<.*>)*')

In [ ]:
def clean_row(row):
  cleaned_row = []
  for i in range(len(row)):
    if i != 32:
      try:
        cleaned_row.append(float(re.sub(units_cleaner_tag , r'\g<1>' , row[i].replace(',','') , )))
      except:
        cleaned_row.append(row[i])
    else:
      cleaned_row.append(row[i])
  return cleaned_row

In [ ]:
cleaned_data = []
for i , row in data_raw.iterrows():
  cleaned_data.append(clean_row(row))

In [ ]:
data_clean = pd.DataFrame(cleaned_data , columns = data_raw.columns)
try:
  data_clean.drop(columns = ['Unnamed: 0'] , inplace=True) # Remove an error I made when saving scraped data
except:
  pass

In [ ]:
for i in range(0,len(data_clean)):
    path = "data/images/"+str(data_clean['name'][i]).replace(" ","_").replace("/","_")
    try:
        if len(os.listdir(path)) == 0:
            os.rmdir(path)
            data_clean.drop(axis = "index" , index=i , inplace=True)
        else:
            pass
    except:
        # print("No dir named" , path)
        data_clean.drop(axis = "index" , index=i , inplace=True)

data_clean.to_csv("boat_data_cleaned.csv" , index=False)


In [ ]:
"data/images/"+str('COOPER 416 TM').replace(" ","_").replace("/","_")

In [ ]:
data_clean.index[(data_clean['name'].eq('49ERFX'))].tolist()[0]

In [ ]:
data_clean.__len__()

In [ ]:
pd.factorize(data_clean['Hull Type'])

# Boat24

In [ ]:
boat24_data['name'].astype('category').cat.codes.plot(kind='hist')